In [1]:
import numpy as np
import pandas as pd

In [2]:
fund = 100
leverage = 3

key_delt = "Del (%)"

# portfolio worth
key_norm = "Nrm"
key_3xn = "3x"
key_lvg = "Lvg"

# percentage gain
key_pctn = "Nrm (pct)"
key_pct3 = "3x (pct)"
key_lvgp = "Lvg (pct)"

# dollar gain
key_netn = "Nrm (USD)"
key_net3 = "3x (USD)"
key_netl = "Lvg (USD)"

In [3]:
def Simulate_Path(deltas):
    df = pd.DataFrame(columns = [key_delt,
                                 key_norm, key_netn, key_pctn, 
                                 key_3xn, key_net3, key_pct3,
                                 key_lvg, key_netl, key_lvgp])
    df = df.append({
            key_norm : fund,
            key_3xn : fund * leverage,
            key_lvg : fund
        },ignore_index = True)

    for delta in deltas:
        # portfolio worth on next day, without leverage
        nxt_n = df.iloc[-1][key_norm] * (1 + delta)
        
        # portfolio worth on next day, with three times fund
        nxt_l = df.iloc[-1][key_3xn] * (1 + delta)
        
        # portfolio worth on next day, with 3x leverage
        nxt_p = df.iloc[-1][key_lvg] * (1 + leverage * delta)
        
        df = df.append({
            key_delt : 100 * delta,
            
            key_norm : nxt_n,
            key_netn : nxt_n - df.iloc[-1][key_norm],
            key_pctn : 100 * (nxt_n - fund) / fund,
            
            key_3xn : nxt_l,
            key_net3 : nxt_l - df.iloc[-1][key_3xn],
            key_pct3 : 100 * (nxt_l - fund * leverage) / (fund * leverage),
            
            key_lvg : nxt_p,
            key_netl : nxt_p - df.iloc[-1][key_lvg],
            key_lvgp : 100 * (nxt_p - fund) / fund 
        }, ignore_index = True)
    return df

## Deterministic Deltas
Non-traditional ETFs do manage to meet their daily target (2:1 or 3:1), but since they "rebalance" daily, the long-term results may be skewed. Say you have bought a 3:1-leveraged ETF that targets thrice the return of index XY. You pay \$100 to buy a share of ETF when the benchmark index is at 10,000. If index XY zooms up 10% the next day to 11,000, the leveraged ETF would increase 30% to \$130. Now, if the index falls from 11,000 back down to 10,000 the following day, there is a 9.09% decline. The leveraged ETF you are holding would __go down thrice, i.e. by 27.27%__. Now, although the index returned to the starting point, a 27.27% decline __from \$130__ would leave you with an ETF share worth just \$94.55, i.e. the ETF share is down by 5.45%.

Read more: A Guide For Buying ETFs On Margin | Investopedia https://www.investopedia.com/articles/investing/060314/guide-buying-etfs-margin.asp#ixzz5NhiSeriw 
Follow us: Investopedia on Facebook

In [4]:
Simulate_Path([0.1, -0.0909])

,Del (%),Nrm,Nrm (USD),Nrm (pct),3x,3x (USD),3x (pct),Lvg,Lvg (USD),Lvg (pct)
0,NaN,100.000,NaN,NaN,300.000,NaN,NaN,100.000,NaN,NaN
1,10.00,110.000,10.000,10.000,330.000,30.000,10.000,130.000,30.000,30.000
2,-9.09,100.001,-9.999,0.001,300.003,-29.997,0.001,94.549,-35.451,-5.451


## Normal Random Deltas 

In [5]:
mu = 0.0001
sigma = 0.01
days = 20
deltas = np.random.normal(mu, sigma, days)
Simulate_Path(deltas)

,Del (%),Nrm,Nrm (USD),Nrm (pct),3x,3x (USD),3x (pct),Lvg,Lvg (USD),Lvg (pct)
0,NaN,100.000000,NaN,NaN,300.000000,NaN,NaN,100.000000,NaN,NaN
1,0.499254,100.499254,0.499254,0.499254,301.497762,1.497762,0.499254,101.497762,1.497762,1.497762
2,0.601129,101.103384,0.604130,1.103384,303.310152,1.812390,1.103384,103.328159,1.830397,3.328159
3,-1.603994,99.481692,-1.621692,-0.518308,298.445076,-4.865076,-0.518308,98.356028,-4.972131,-1.643972
4,0.434206,99.913648,0.431956,-0.086352,299.740944,1.295867,-0.086352,99.637232,1.281204,-0.362768
5,0.891236,100.804115,0.890467,0.804115,302.412345,2.671401,0.804115,102.301242,2.664010,2.301242
6,0.317722,101.124392,0.320277,1.124392,303.373176,0.960832,1.124392,103.276344,0.975102,3.276344
7,0.821700,101.955331,0.830939,1.955331,305.865993,2.492816,1.955331,105.822208,2.545864,5.822208
8,-0.323943,101.625054,-0.330277,1.625054,304.875161,-0.990832,1.625054,104.793797,-1.028411,4.793797
9,-0.331515,101.288151,-0.336902,1.288151,303.864454,-1.010707,1.288151,103.751575,-1.042222,3.751575
